# 💬 ChipChat - 데이터시트 챗봇

**전처리된 데이터시트 JSON 파일을 기반으로 질의응답을 수행하는 챗봇입니다.**

---

## 🚀 시작하기 전에

이 노트북은 ChipChat 앱을 실행하는 방법을 제공합니다.
로컬 환경과 Google Colab 환경 모두에서 사용할 수 있습니다.

## ✨ 주요 기능들
- 🤖 **AI Agent 시스템**: LangGraph 기반 스마트 에이전트
- 🔧 **3가지 Tool 자동 선택**: chipDB 검색, 벡터스토어 검색, PDF 처리
- 📊 **ChipDB.csv 연동**: 부품 사양 요약 데이터베이스 활용
- 🎯 **다중 LLM 지원**: OpenAI와 Claude 모델 선택 가능
- 📄 **실시간 PDF 업로드**: 새 데이터시트 자동 처리 및 통합
- 🔍 **고급 필터링**: 부품번호, 제조사, 카테고리별 검색
- 🛠️ **프롬프트 커스터마이징**: 시스템 프롬프트 자유 수정
- 🏷️ **메타데이터 추적**: 소스 정보 표시

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: 환경 설정 { display-mode: "form" }
#@markdown 로컬 또는 Google Drive 환경을 선택하고 설정을 관리합니다.

import os
import json
from pathlib import Path

# 환경 선택 토글
use_google_drive = False #@param {type:"boolean"}
#@markdown ✅ 체크: Google Drive 사용 (Colab 환경) | ❌ 체크해제: 로컬 환경 사용

# config.json 파일 경로
config_file = Path('prep/config.json')

if config_file.exists():
    # 기존 config.json 로드
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # 환경 설정 업데이트
    config['environment']['use_google_drive'] = use_google_drive
    
    # 업데이트된 설정 저장
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=4, ensure_ascii=False)
    
    print("✅ config.json 업데이트 완료!")
else:
    print("❌ config.json 파일이 없습니다. 기본 설정을 생성합니다.")
    
    # 기본 config.json 생성
    default_config = {
        "environment": {
            "use_google_drive": use_google_drive,
            "description": "Environment selection: true for Google Drive (Colab), false for local"
        },
        "paths": {
            "google_drive": {
                "base_path": "/content/drive/MyDrive",
                "prep_json_folder": "/content/drive/MyDrive/prep_json",
                "vectorstore_folder": "/content/drive/MyDrive/vectorstore",
                "prompt_templates_folder": "/content/drive/MyDrive/prompts",
                "model_cache_folder": "/content/drive/MyDrive/hf_model_cache",
                "logs_folder": "/content/drive/MyDrive/chipchat_logs",
                "prep_datasheets_folder": "/content/drive/MyDrive/datasheets",
                "prep_prep_json_folder": "/content/drive/MyDrive/prep_json",
                "prep_vectorstore_folder": "/content/drive/MyDrive/vectorstore"
            },
                    "local": {
            "base_path": ".",
            "prep_json_folder": "./prep/prep_json",
            "vectorstore_folder": "./prep/vectorstore",
            "prompt_templates_folder": "./prompts",
            "model_cache_folder": "./hf_model_cache",
            "logs_folder": "./logs",
            "prep_datasheets_folder": "./prep/datasheets",
            "prep_prep_json_folder": "./prep/prep_json",
            "prep_vectorstore_folder": "./prep/vectorstore"
        }
        },
        "vectorstore": {
            "default_name": "datasheet_vectors_final"
        },
        "models": {
            "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
            "supported_llm": {
                "openai": [
                    "gpt-4o-mini",
                    "gpt-4o",
                    "gpt-3.5-turbo"
                ],
                "claude": [
                    "claude-3-sonnet-20240229",
                    "claude-3-haiku-20240307",
                    "claude-3-opus-20240229"
                ]
            }
        },
        "pdf_processing": {
            "pages_per_chunk": 3,
            "chunk_overlap": 1,
            "output_formats": {
                "save_filtered_pdf": True,
                "save_summary_only": False,
                "save_combined": True
            }
        }
    }
    
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(default_config, f, indent=4, ensure_ascii=False)
    
    config = default_config
    print("✅ 기본 config.json 생성 완료!")

# 현재 환경 설정 표시
env_type = "Google Drive" if use_google_drive else "로컬"
current_paths = config['paths']['google_drive' if use_google_drive else 'local']

print(f"\n🌐 {env_type} 환경으로 설정되었습니다!")
print(f"📋 설정된 경로들:")
for key, path in current_paths.items():
    if key != 'base_path':
        print(f"  • {key.replace('_', ' ').title()}: {path}")

# Google Drive 마운트 (필요한 경우)
if use_google_drive:
    try:
        from google.colab import drive
        print("\n🔄 Google Drive 마운트 중...")
        drive.mount('/content/drive')
        print("✅ Google Drive 연동 완료!")
    except ImportError:
        print("⚠️ Google Colab 환경이 아닙니다. 환경 설정을 로컬로 변경합니다.")
        config['environment']['use_google_drive'] = False
        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=4, ensure_ascii=False)
        current_paths = config['paths']['local']
        print("💻 로컬 환경으로 전환되었습니다.")

# 필요한 디렉토리 생성
print("\n📁 필요한 디렉토리 생성 중...")
for key, path_str in current_paths.items():
    if key != 'base_path':
        path = Path(path_str)
        path.mkdir(parents=True, exist_ok=True)
        print(f"  ✅ {path}")

print("\n🎉 환경 설정이 완료되었습니다!")


In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
import subprocess
from pathlib import Path
import json

# config.json에서 환경 설정 읽기
config_file = Path('prep/config.json')
use_google_drive = False

if config_file.exists():
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    use_google_drive = config.get('environment', {}).get('use_google_drive', False)

print("📥 필요한 라이브러리 설치 중...")

# 환경별 설치 과정
if use_google_drive:
    print("🌐 Google Colab 환경 설정...")
    
    # GitHub 저장소 클론 (Colab 환경)
    try:
        if not Path('src').exists():  # src 폴더가 없으면 클론 필요
            print("🔄 GitHub 저장소 클론 중...")
            subprocess.run(['git', 'clone', 'https://github.com/doyoung42/chipchat_demo.git', 'temp_clone'], check=True)
            
            # 필요한 파일들 복사
            import shutil
            if Path('temp_clone/src').exists():
                shutil.copytree('temp_clone/src', 'src', dirs_exist_ok=True)
            if Path('temp_clone/requirements.txt').exists():
                shutil.copy2('temp_clone/requirements.txt', 'requirements.txt')
            
            # 임시 폴더 삭제
            shutil.rmtree('temp_clone')
            print("✅ GitHub 저장소 클론 및 파일 복사 완료")
        else:
            print("✅ 소스 코드 이미 존재")
    except Exception as e:
        print(f"❌ GitHub 클론 실패: {str(e)}")
    
    # Colab 전용 패키지 설치
    try:
        subprocess.run(['pip', 'install', 'pyngrok==7.0.1', '-q'], check=True)
        print("✅ pyngrok (외부 접속용) 설치 완료")
    except Exception as e:
        print(f"❌ pyngrok 설치 실패: {str(e)}")

else:
    print("💻 로컬 환경 설정...")
    
    # 로컬 환경에서는 이미 git clone되어 있다고 가정
    if Path('src').exists():
        print("✅ 소스 코드 확인 완료 (로컬 저장소)")
    else:
        print("❌ src 폴더를 찾을 수 없습니다.")
        print("💡 GitHub에서 저장소를 clone했는지 확인해주세요.")
        print("   git clone https://github.com/doyoung42/chipchat_demo.git")

# requirements.txt 설치
if use_google_drive:
    if Path('requirements.txt').exists():
        try:
            subprocess.run(['pip', 'install', '-r', 'requirements.txt', '-q'], check=True)
            print("✅ Requirements 설치 완료")
        except Exception as e:
            print(f"❌ Requirements 설치 실패: {str(e)}")
    else:
        print("⚠️ requirements.txt 파일을 찾을 수 없습니다.")
        print("💡 기본 라이브러리들만 사용합니다.")

# Python path 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

# 새로운 디렉토리 구조 추가
src_dir = current_dir / 'src'
if src_dir.exists():
    sys.path.append(str(src_dir))
    for subdir in ['config', 'models', 'utils', 'app']:
        subdir_path = src_dir / subdir
        if subdir_path.exists():
            sys.path.append(str(subdir_path))

print("\n✅ 라이브러리 설치 및 경로 설정 완료!")
print("\n📋 지원되는 LLM 모델:")
print("🔸 OpenAI: gpt-4o-mini, gpt-4o, gpt-3.5-turbo")
print("🔸 Claude: claude-3-sonnet, claude-3-haiku, claude-3-opus")

In [ ]:
#@title 🚀 2-1단계: 로깅 시스템 초기화 및 모델 사전 다운로드 { display-mode: "form" }
#@markdown HuggingFace 모델을 캐싱하여 빠른 로딩을 지원합니다.

import time
from src.utils.logger import get_logger
from src.utils.model_cache import get_model_cache

# 로깅 시스템 초기화
print("📊 로깅 시스템 초기화 중...")
logger = get_logger()
logger.log_system_info()
logger.info("Main notebook 실행 시작")

# 모델 캐시 시스템 초기화
print("\n🗄️ 모델 캐시 시스템 초기화 중...")
model_cache = get_model_cache()
cache_info = model_cache.get_cache_info()

print(f"📁 캐시 디렉토리: {cache_info['cache_dir']}")
print(f"📦 캐시된 모델 수: {cache_info['model_count']}")
print(f"💾 전체 캐시 크기: {cache_info['total_size_mb']:.2f} MB")

if cache_info['models']:
    print("\n✅ 캐시된 모델:")
    for model in cache_info['models']:
        print(f"  - {model}")

# 모델 사전 다운로드 옵션
download_model = True #@param {type:"boolean"}
model_name = "sentence-transformers/all-MiniLM-L6-v2" #@param {type:"string"}

if download_model:
    print(f"\n🔄 모델 다운로드 또는 캐시 확인: {model_name}")
    
    @logger.measure_time("모델 준비")
    def prepare_model():
        # 캐시 확인
        if model_cache.is_model_cached(model_name):
            print("✅ 모델이 이미 캐시되어 있습니다.")
            # 캐시에서 로드
            if model_cache.load_model_from_cache(model_name):
                print("✅ 캐시에서 모델 로드 완료!")
                return True
        
        # 캐시에 없으면 다운로드
        print("📥 모델을 다운로드합니다...")
        try:
            from langchain_huggingface import HuggingFaceEmbeddings
            
            # 모델 다운로드 (초기화를 통해)
            start_time = time.time()
            embeddings = HuggingFaceEmbeddings(
                model_name=model_name,
                model_kwargs={'device': 'cpu'}
            )
            download_time = time.time() - start_time
            
            print(f"✅ 모델 다운로드 완료! (소요 시간: {download_time:.2f}초)")
            
            # 캐시에 저장
            print("💾 모델을 캐시에 저장 중...")
            if model_cache.save_model_to_cache(model_name):
                pass  # save_model_to_cache에서 이미 저장 위치 정보를 출력함
            else:
                print("⚠️ 모델 캐시 저장 실패 (다음 실행 시 다시 다운로드됩니다)")
            
            return True
            
        except Exception as e:
            logger.error(f"모델 준비 실패: {e}")
            print(f"❌ 모델 준비 실패: {e}")
            return False
    
    # 모델 준비 실행
    success = prepare_model()
    
    if success:
        print("\n🎉 모델 준비 완료! 이제 Streamlit 앱이 더 빠르게 로드됩니다.")
    else:
        print("\n⚠️ 모델 준비에 실패했습니다. 하지만 계속 진행할 수 있습니다.")

# 성능 요약 표시
print("\n📊 현재까지의 성능 요약:")
perf_summary = logger.get_performance_summary()
for op, stats in perf_summary.items():
    print(f"\n🔹 {op}:")
    print(f"   - 평균 시간: {stats['avg_time']:.2f}초")
    print(f"   - 성공: {stats['success_count']}회, 실패: {stats['fail_count']}회")

print("\n✅ 모든 준비가 완료되었습니다!")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown AI 서비스의 API 키를 입력하여 챗봇을 설정합니다.

import os
import json
from pathlib import Path

openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # 환경 변수 설정
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("✅ OpenAI API 키 설정 완료!")
        
    if claude_api_key:
        os.environ["ANTHROPIC_API_KEY"] = claude_api_key
        print("✅ Claude API 키 설정 완료!")
    
    if hf_token:
        os.environ["HF_TOKEN"] = hf_token
        print("✅ HuggingFace 토큰 설정 완료!")
    
    # Streamlit 시크릿 파일 생성
    secrets_dir = Path(".streamlit")
    secrets_dir.mkdir(exist_ok=True)
    
    secrets = {}
    if openai_api_key:
        secrets["openai_api_key"] = openai_api_key
    if claude_api_key:
        secrets["anthropic_api_key"] = claude_api_key
    if hf_token:
        secrets["hf_token"] = hf_token
    
    with open(secrets_dir / "secrets.toml", "w") as f:
        for key, value in secrets.items():
            f.write(f'{key} = "{value}"\n')
    
    # 토큰 저장
    try:
        from src.config.token_manager import TokenManager
        token_manager = TokenManager()
        
        if openai_api_key:
            token_manager.set_token('openai', openai_api_key)
        
        if claude_api_key:
            token_manager.set_token('anthropic', claude_api_key)
        
        if hf_token:
            token_manager.set_token('huggingface', hf_token)
            
        print("✅ 토큰 관리자에 API 키 저장 완료!")
    except Exception as e:
        print(f"❌ 토큰 저장 중 오류가 발생했습니다: {str(e)}")
    
    # 프롬프트 시스템 초기화 (이제 PromptManager가 자동으로 처리)
    print("✅ 프롬프트 시스템이 초기화되었습니다.")

In [ ]:
#@title 📊 4단계: Streamlit 서버 실행 (로컬 최적화) { display-mode: "form" }
#@markdown 로컬 환경에 최적화된 간단한 Streamlit 실행

import subprocess
import time
import os
import json
from pathlib import Path

# 환경 확인
config_file = Path('prep/config.json')
is_colab_env = False

if config_file.exists():
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    is_colab_env = config.get('environment', {}).get('use_google_drive', False)

# 앱 설정
app_file = "src/app/streamlit_app.py"
port = 8501

print(f"🚀 ChipChat {'(Google Colab)' if is_colab_env else '(로컬)'} 시작 중...")
print(f"📁 파일: {app_file}")
print(f"🌐 포트: {port}")
print(f"✨ 주요 기능: AI 에이전트, 멀티턴 대화, LLM 모델 선택")

# 환경별 처리
if not is_colab_env:
    # 로컬 환경 (간소화)
    print("\n💻 로컬 환경에서 실행합니다.")
    
    # 기존 프로세스 정리 (조용히)
    try:
        subprocess.run(['pkill', '-f', 'streamlit'], 
                      check=False, capture_output=True, text=True)
        time.sleep(1)
        print("✅ 기존 프로세스 정리 완료")
    except:
        pass
    
    # 간단한 Streamlit 명령어
    cmd = [
        "streamlit", "run", 
        app_file,
        f"--server.port={port}", 
        "--server.address=localhost",
        "--server.headless=true",
        "--browser.gatherUsageStats=false",
        "--logger.level=warning"  # 로그 최소화
    ]
    
    print("🔧 Streamlit 시작 중...")
    
    # 로컬 환경에서도 기본 경로 설정 (오류 방지용)
    try:
        from src.utils.config_manager import get_config_manager
        config = get_config_manager()
        vectorstore_folder = config.get_path('vectorstore_folder')
        prep_json_folder = config.get_path('prep_json_folder')
        prompt_templates_folder = config.get_path('prompt_templates_folder')
    except:
        vectorstore_folder = './vectorstore'
        prep_json_folder = './prep_json'
        prompt_templates_folder = './prompt_templates'

    # 환경 변수 설정
    os.environ['VECTORSTORE_PATH'] = str(vectorstore_folder)
    os.environ['JSON_FOLDER_PATH'] = str(prep_json_folder)
    os.environ['PROMPT_TEMPLATES_PATH'] = str(prompt_templates_folder)
    
    try:
        # 백그라운드 실행 (로그 최소화)
        process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # 잠시 대기
        time.sleep(3)
        
        # 상태 확인
        if process.poll() is None:  # 실행 중
            print("✅ ChipChat이 성공적으로 시작되었습니다!")
            print(f"\n🔗 브라우저에서 다음 주소로 접속하세요:")
            print(f"🌐 http://localhost:{port}")
            print(f"\n💡 사용 방법:")
            print(f"  • 설정 페이지에서 LLM 모델을 선택하세요")
            print(f"  • AI 에이전트가 자동으로 도구를 선택합니다")
            print(f"  • 종료하려면 5단계를 실행하세요")
            
            # 프로세스 저장 (5단계에서 사용)
            globals()['streamlit_process'] = process
            
        else:
            print("❌ Streamlit 시작에 실패했습니다.")
            print("💡 다음을 확인해보세요:")
            print("  • requirements.txt가 설치되었는지")
            print("  • API 키가 설정되었는지")
            print("  • src 폴더가 존재하는지")
            
    except Exception as e:
        print(f"❌ 실행 중 오류: {e}")
        print("💡 수동으로 실행해보세요:")
        print(f"   streamlit run {app_file} --server.port={port}")

else:
    # Google Colab 환경 (기존 복잡한 로직 실행)
    print("🌐 Google Colab 환경에서 실행합니다.")
    print("⚠️ Google Colab 기능은 현재 버전에서 간소화되었습니다.")
    print("💡 수동으로 실행하거나 이전 버전을 사용해주세요.")
    
    # 기본 변수 설정 (오류 방지용)
    vectorstore_folder = './vectorstore'
    prep_json_folder = './prep_json'
    prompt_templates_folder = './prompt_templates'
    
    # 프로세스 저장 (5단계에서 사용)
    globals()['streamlit_process'] = None
    
    print("💡 Google Colab에서 수동 실행하려면:")
    print(f"   streamlit run {app_file} --server.port={port}")

# 4단계 완료 - 환경별 분기로 처리됨

In [ ]:
#@title 🛑 5단계: 서버 중지 { display-mode: "form" }
#@markdown 작업을 마치면 서버를 중지합니다.

import os
import signal
import subprocess
import time

# 서버 중지 여부 확인
stop_server = True  #@param {type:"boolean"}

if stop_server:
    print("🛑 서버 종료 프로세스를 시작합니다...")
    
    # ngrok 터널 종료 (ngrok 사용한 경우에만)
    try:
        from pyngrok import ngrok
        ngrok.kill()
        print("✅ ngrok 터널이 종료되었습니다.")
    except ImportError:
        pass
    except Exception as e:
        print(f"ℹ️ ngrok 종료: {str(e)}")
    
    # 4단계에서 생성된 프로세스 먼저 종료
    terminated_process = False
    if 'streamlit_process' in globals():
        try:
            process = globals()['streamlit_process']
            if process and process.poll() is None:
                print("🔄 4단계에서 실행된 Streamlit 프로세스를 종료 중...")
                process.terminate()
                time.sleep(3)
                
                if process.poll() is None:
                    print("⚡ 강제 종료를 시도합니다...")
                    process.kill()
                    time.sleep(2)
                
                print("✅ Streamlit 프로세스가 정상적으로 종료되었습니다.")
                terminated_process = True
            else:
                print("ℹ️ Streamlit 프로세스가 이미 종료되었습니다.")
        except Exception as e:
            print(f"⚠️ 프로세스 종료 중 오류: {e}")
    
    # 추가적으로 모든 streamlit 프로세스 종료
    try:
        print("🔍 남은 Streamlit 프로세스를 검색 중...")
        result = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
        if result.stdout.strip():
            pids = result.stdout.strip().split('\n')
            print(f"📋 발견된 Streamlit 프로세스: {len(pids)}개")
            
            subprocess.run(['pkill', '-f', 'streamlit'], check=False)
            time.sleep(2)
            
            result_after = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
            if not result_after.stdout.strip():
                print("✅ 모든 Streamlit 프로세스가 정리되었습니다.")
            else:
                print("⚠️ 일부 프로세스가 여전히 실행 중일 수 있습니다.")
        else:
            print("✅ 실행 중인 Streamlit 프로세스가 없습니다.")
            
    except Exception as e:
        print(f"⚠️ 프로세스 정리 중 오류가 발생했습니다: {e}")
        print("💡 수동으로 런타임을 재시작하여 서버를 종료할 수 있습니다.")
    
    # 포트 사용 상태 확인
    try:
        import socket
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('localhost', 8501))
        sock.close()
        
        if result != 0:
            print("✅ 포트 8501이 해제되었습니다.")
        else:
            print("⚠️ 포트 8501이 아직 사용 중입니다.")
    except Exception as e:
        print(f"ℹ️ 포트 상태 확인 실패: {e}")
    
    print("\n📋 서버 종료 완료!")
    print("💡 완전한 정리를 위해 런타임을 재시작하는 것을 권장합니다.")
    print("🔄 런타임 → 세션 재시작을 통해 모든 프로세스를 완전히 종료할 수 있습니다.")
else:
    print("ℹ️ 서버 중지가 취소되었습니다.")
    print("💡 서버를 중지하려면 위의 체크박스를 선택하고 다시 실행해주세요.")

## 🛠️ 문제 해결

**애플리케이션이 실행되지 않는 경우:**

1. **API 키 확인**: OpenAI API 키가 올바르게 입력되었는지 확인
2. **세션 재시작**: 런타임 → 세션 재시작 후 처음부터 다시 실행
3. **JSON 파일 확인**: 전처리된 JSON 파일이 올바른 경로에 존재하는지 확인

**사용 중 문제가 발생하는 경우:**
- 브라우저를 새로고침하세요
- 네트워크 연결 상태를 확인하세요
